In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

1. 각 회원상태에 대해 codename들에 대한 count를 해주세요

2. 날짜별로 point를 가장 많이 획득한 level을 보여주세요.

3. pointnm 별로 획득한 point의 종류를 언더바로 이어서 보여주세요

In [7]:
item = spark.read.parquet("/content/item_his.parquet")
member = spark.read.parquet('/content/member.parquet')
point = spark.read.parquet("/content/point_his.parquet")
regdate = spark.read.parquet("/content/regdate_his.parquet")
study = spark.read.parquet("/content/study_his.parquet")

In [17]:
# 1. 각 회원상태에 대해 codename들에 대한 count를 해주세요
# 회원상태 -> member  /  codename -> item

member.printSchema(), item.printSchema()

root
 |-- idx: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade: string (nullable = true)

root
 |-- idx: long (nullable = true)
 |-- lv: double (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: long (nullable = true)
 |-- price: long (nullable = true)



(None, None)

In [12]:
item_member = item.join(member, ['idx'])
item_member.show(5)

+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|sex|  status|  grade|
+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50| 남|유료회원|  0학년|
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50| 남|유료회원|초1학년|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50| 남|유료회원|  0학년|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50| 남|유료회원|초1학년|
|20163|161.0| 202304|20230408|  액세서리|아바타파츠구분|      50|   50| 여|유료회원|초2학년|
+-----+-----+-------+--------+----------+--------------+--------+-----+---+--------+-------+
only showing top 5 rows



In [14]:
item_group = item_member.groupby('status', 'codename').agg(F.count(F.col('codename')).alias('codecount'))

In [ ]:
item_group.orderBy('status', 'codename').show(50)

In [18]:
# 2. 날짜별로 point를 가장 많이 획득한 level을 보여주세요.
# point_his -> point  /  item_his -> lv

point.printSchema(), item.printSchema()

root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- point: long (nullable = true)

root
 |-- idx: long (nullable = true)
 |-- lv: double (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: long (nullable = true)
 |-- price: long (nullable = true)



(None, None)

In [19]:
point.count(), item.count()

(92079, 109545)

In [23]:
point_time = point.join(item, ['idx', 'proc_ymd'])

point_time.show(5)

+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
|  idx|proc_ymd|proc_ym|point|  lv|proc_ym|  codename| mascodename|payprice|price|
+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
|88572|20230408| 202304| 5000|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 4500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 3000|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 3500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
|88572|20230408| 202304| 2500|14.0| 202304|상태메시지|기타파츠구분|      50|   50|
+-----+--------+-------+-----+----+-------+----------+------------+--------+-----+
only showing top 5 rows



In [27]:
point_group = point_time.groupby('proc_ymd', 'lv').agg(F.sum(F.col('point')).alias('point_sum'))

In [28]:
point_group.orderBy('proc_ymd', 'lv').show(5)

+--------+----+---------+
|proc_ymd|  lv|point_sum|
+--------+----+---------+
|20230401| 2.0|     2000|
|20230401| 4.0|     9000|
|20230401| 6.0|    28000|
|20230401| 8.0|     6500|
|20230401|10.0|    52500|
+--------+----+---------+
only showing top 5 rows



In [33]:
window_var = W.Window.partitionBy('proc_ymd').orderBy(F.desc('point_sum'))

point_window = point_group.withColumn('point_rank', F.row_number().over(window_var))

# point_window.printSchema()
point_window.filter(F.col('point_rank') == 1).show()

+--------+----+---------+----------+
|proc_ymd|  lv|point_sum|point_rank|
+--------+----+---------+----------+
|20230401|14.0|   138000|         1|
|20230402|11.0|    28500|         1|
|20230403|13.0|    57500|         1|
|20230404|10.0|   148000|         1|
|20230405|15.0|   202500|         1|
|20230406|10.0|   193500|         1|
|20230407| 6.0|   302000|         1|
|20230408|13.0|   113500|         1|
|20230409|11.0|   240000|         1|
|20230410|13.0|    68000|         1|
|20230411| 9.0|   179000|         1|
|20230412|19.0|   109500|         1|
|20230413|11.0|   317000|         1|
|20230414|11.0|   205000|         1|
|20230415|12.0|   182000|         1|
|20230416|16.0|    49000|         1|
|20230417|11.0|   146500|         1|
|20230418|23.0|   137500|         1|
|20230419|11.0|   135000|         1|
|20230420|14.0|   687000|         1|
+--------+----+---------+----------+
only showing top 20 rows



In [36]:
# 3. pointnm 별로 획득한 point의 종류를 언더바로 이어서 보여주세요
# pointnm -> study  /  point -> point

study.printSchema(), point.printSchema()

root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- pointnm: string (nullable = true)

root
 |-- idx: long (nullable = true)
 |-- proc_ym: long (nullable = true)
 |-- proc_ymd: long (nullable = true)
 |-- point: long (nullable = true)



(None, None)

In [38]:
study_point = study.join(point, ['idx', 'proc_ymd'])

study_point.show(5)

+-----+--------+-------+-----------+-------+-----+
|  idx|proc_ymd|proc_ym|    pointnm|proc_ym|point|
+-----+--------+-------+-----------+-------+-----+
|94954|20230611| 202306|한글 스피치| 202306| 5000|
|94954|20230611| 202306|한글 스피치| 202306| 4000|
|94954|20230611| 202306|한글 스피치| 202306| 4500|
|89734|20230418| 202304|한글 스피치| 202304| 5000|
|89734|20230418| 202304|한글 스피치| 202304| 5000|
+-----+--------+-------+-----------+-------+-----+
only showing top 5 rows



In [49]:
study_point = study_point.groupby('pointnm').agg(F.collect_set(F.col('point')).alias('point_set'))

In [52]:
data1 = [3000, 3500, 1000, 500, 4500]

def underbar(var):

  str_var = [str(i) for i in data1]

  return_var = '_'.join(str_var)

  return return_var

underbar(data1)

'3000_3500_1000_500_4500'

In [53]:
underbar_udf = F.udf(underbar)

In [58]:
study_point.withColumn('point_udf', underbar_udf(F.col('point_set'))).show(truncate=False)

+-----------------+-----------------------------------------------------------+-----------------------+
|pointnm          |point_set                                                  |point_udf              |
+-----------------+-----------------------------------------------------------+-----------------------+
|일기 쓰기        |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|학교 체험        |[5000]                                                     |3000_3500_1000_500_4500|
|중학 특강        |[500, 1500]                                                |3000_3500_1000_500_4500|
|AI 영어          |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|도전! AI 받아쓰기|[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|학교 공부 맛보기 |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_500_4500|
|중학 신입생      |[3000, 3500, 1000, 4500, 5000, 2500, 500, 4000, 1500, 2000]|3000_3500_1000_50